# Star-Wars Aura Demo Notebook 
## Cypher Examples Using Neo4j Python Driver


## Overview
To demonstrate how to connnect to a Neo4j instance running in Aura.  This uses the Neo4j Driver version
### Created: September 11, 2022
### Author:  Mark Quinsland

### Additional resources:
Some of this material is based on the excellent, highly-recommended "Bite-Sized Neo4j for Data Scientiests" series of notebooks and youtube clips by CJ Sullivan. 

#### Note on Python Drivers & Python Client:
For some of the notebooks in CJ's series, the official Neo4j Python driver named 'neo4j' is used.  In others, the community-supported 'py2neo' driver is used.  This notebook uses the 'neo4j' driver.  Since the publishin of the Bite-Sized series, Neo4j released a new Python client named 'graphdatascience' that supports additional Graph Data Science features.  This client should only be used to access Neo4j servers where the GDS library has been installed.    Examples using the new graphdatascience library are available from the github repo containing this file.  

https://github.com/cj2001/bite_sized_data_science

https://youtu.be/Niys6g6NFfw?list=PL9Hl4pk2FsvVShoT5EysHcrs-hyCsXaWC

https://medium.com/@cj2001



In [17]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import pandas as pd
import time


In [18]:
#aura connection details

uri="bolt://localhost"
user = 'neo4j'
password = 'jedi'

uri="neo4j+s://3c6eb2f6.databases.neo4j.io"


In [19]:
conn = Neo4jConnection(uri=uri, user=user, pwd=password)

In [20]:
## simple example to test connection & return results
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
# print(result)
# Consume the result object
print(result[0]['ct'])

326


### Cypher Query Examples

#### Execute Cypher read query and process results

In [21]:
# load any parameters 

params = {'name': 'Han Solo' 
         }

query = """MATCH (p:Character {name: $name})-[i:INTERACTED_WITH]-(p2:Character)
           RETURN  p2.name AS otherName, i.count as interactionCount
           ORDER BY interactionCount DESC LIMIT 10
           
"""


result = conn.query(query, parameters=params)

for row in result:
    print (row['otherName'], row['interactionCount'])

#### Execute Cypher read query and export to DataFrame

In [25]:
# load any parameters 

params = {'name': 'Han Solo'  }

query = """MATCH (p:Character {name: $name})-[i:INTERACTED_WITH]-(p2:Character)
           RETURN  p2.name AS otherName, i.count as interactionCount
           ORDER BY interactionCount DESC LIMIT 10         
"""

# run query to get results
result = conn.query(query, parameters=params)
 
# create a dataframe from Cypher Resultset    
dtf_data = pd.DataFrame([dict(_) for _ in result])
dtf_data.head(3)


""


## Write Data to Neo4j using DataFrame

In [24]:
def add_users_using_df(rows):
    
    query = """UNWIND $rows AS row
               MERGE (u:User {id: row.id})
               SET u.name = row.name 
               RETURN COUNT(*) AS total
            """
    
    return insert_data(query, rows)


user_data = {
    'id': [0, 1, 2, 3, 4],
    'name': ['Alice', 'Brian', 'Carla', 'David', 'Ed']
}

user_df = pd.DataFrame(user_data)
user_df.head()

add_users_using_df(user_df)

{'total': 5, 'batches': 1, 'time': 0.1604321002960205}

### Utility method for inserting records into Neo using batches

In [8]:
def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):

        res = conn.query(query, 
                         parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        
    return result

### Utility class for Neo4j connection

In [6]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response